In [ ]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

file_path = '/content/drive/MyDrive/df_combined_feb.csv'
df_combined = pd.read_csv(file_path)
print(df_combined.head())



                                        comment_text  toxic
0  It is worth it. Only expose my past. All time ...      1
1  the good sense not use you. you are the real v...      1
2  In regard to what I posted and you've already ...      1
3  But what you're saying!??? This is the version...      1
4  Nihilo remains committed to qualify to García ...      1


In [ ]:
df_combined['toxic'].value_counts()

,count
toxic,
0,1676
1,1310


In [ ]:
df_combined['toxic'].value_counts()

,count
toxic,
0,1676
1,1310


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# Download the wordnet corpus
nltk.download('wordnet') # This line downloads the necessary WordNet corpus
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # 1. find urls and replace them with 'urlplaceholder'
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, 'urlplaceholder', text)

    # 2. Convert to lowercase
    text = text.lower().strip()

    # 3. Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    # 4. Remove stopwords
    stop_words = set(stopwords.words('english'))
    clean_tokens = [word for word in text.split() if word not in stop_words]

    # 6. Lemmatization
    lemmatizer = WordNetLemmatizer()
    # 6.1 Reduce words to their root form
    lemmed = [lemmatizer.lemmatize(w) for w in clean_tokens]
    # 6.2 Lemmatize verbs by specifying pos
    clean_tokens = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]

    cleaned_text = ' '.join(clean_tokens)

    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import time # Import the time module
start = time.time()
df_combined["clean_comment"] = df_combined["comment_text"].apply(clean_text)
end = time.time()
print(f"Execution time: {int((end-start)/60)} minutes and {int((end-start)%60)} seconds")
df_combined.head()

Execution time: 0 minutes and 10 seconds


,comment_text,toxic,clean_comment
0,It is worth it. Only expose my past. All time ...,1,worth expose past time past better much less w...
1,the good sense not use you. you are the real v...,1,good sense use real vandal sooner later quote ...
2,In regard to what I posted and you've already ...,1,regard post already remove maroni find unfair ...
3,But what you're saying!??? This is the version...,1,say version november decline female change mal...
4,Nihilo remains committed to qualify to García ...,1,nihilo remain commit qualify garc oliver leade...


In [ ]:
model = model.to('cuda')

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
X = list(df_combined["clean_comment"])
y = list(df_combined["toxic"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=100)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=100)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(X_train),len(X_val)

(2388, 598)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([  101,  2424,  2428,  5754,  6977,  2755,  3613, 25672,  2689,  2592,
          6135,  6149,  8292, 28228, 12031,  3120,  2394,  3432,  3059,  4839,
          2755,  4748, 10020,  2522,  3166,  5562,  6366,  2592,  6835,  3114,
         10424, 19966, 11657,  5754,  6977,  5236,  4638,  3120,  2156, 17244,
          6679,  2271,  6365,  5796,  2290,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
'''# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)'''

'''args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save at the end of each epoch
    learning_rate=1e-4,  # Increased learning rate for faster convergence
    per_device_train_batch_size=8,  # Reduced batch size
    num_train_epochs=20,  # Increased epochs
    weight_decay=0.001,  # Reduced weight decay
    load_best_model_at_end=True,  # Load best model based on validation loss
    early_stopping_patience=3,  # Early stopping after 3 epochs without improvement
    max_grad_norm=1.0,  # Gradient clipping to avoid instability
    optim="adamw_torch",  # Optimizer choice
    lr_scheduler_type="linear",  # Gradual learning rate decay
    report_to="none",  # Disable W&B reporting if needed''
)
'

'# Define Trainer\nargs = TrainingArguments(\n    output_dir="output",\n    num_train_epochs=1,\n    per_device_train_batch_size=8\n\n)\ntrainer = Trainer(\n    model=model,\n    args=args,\n    train_dataset=train_dataset,\n    eval_dataset=val_dataset,\n    compute_metrics=compute_metrics\n)'

In [ ]:
''''training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,  # Adjusted from 1e-12
    per_device_train_batch_size=12,
    num_train_epochs=14,  # Training can go up to 14 epochs, but early stopping prevents overfitting
    weight_decay=0.01,
    load_best_model_at_end=True,
)''''

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=11,
    per_device_train_batch_size=16,
    report_to="none"  # Disable W&B logging
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

Step,Training Loss
500,0.335600
1000,0.052100


Step,Training Loss
500,0.335600
1000,0.052100
1500,0.007200


TrainOutput(global_step=1650, training_loss=0.11988045304110556, metrics={'train_runtime': 484.5176, 'train_samples_per_second': 54.215, 'train_steps_per_second': 3.405, 'total_flos': 1349883047304000.0, 'train_loss': 0.11988045304110556, 'epoch': 11.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.5800062417984009,
 'eval_accuracy': 0.7976588628762542,
 'eval_precision': 0.7680608365019012,
 'eval_recall': 0.7709923664122137,
 'eval_f1': 0.7695238095238095,
 'eval_runtime': 4.509,
 'eval_samples_per_second': 132.624,
 'eval_steps_per_second': 16.633,
 'epoch': 11.0}

In [ ]:
#trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.9035868644714355,
 'eval_accuracy': 0.802675585284281,
 'eval_precision': 0.8025210084033614,
 'eval_recall': 0.7290076335877863,
 'eval_f1': 0.764,
 'eval_runtime': 2.202,
 'eval_samples_per_second': 271.567,
 'eval_steps_per_second': 34.059,
 'epoch': 14.0}

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "Don’t worry, we all make mistakes. Let’s work together to fix this"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.8635, -4.5433]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[9.9978e-01, 2.2329e-04]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.9997768 , 0.00022329]], dtype=float32)

In [ ]:
#pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0


In [ ]:
# Install Gradio (run this in your terminal)
# pip install gradio

import gradio as gr
import torch
from transformers import AutoTokenizer, BertForSequenceClassification

# Load your tokenizer and custom model
tokenizer = AutoTokenizer.from_pretrained("Final_CustomModel_toxic_feb5")
model_2 = BertForSequenceClassification.from_pretrained("Final_CustomModel_toxic_feb5").to('cuda')

def predict_sentiment(text):
    # Tokenizing the input text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')

    # Getting model predictions
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model_2(**inputs)

    # Applying softmax to get probabilities
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Converting predictions to numpy for easier interpretation
    predictions = predictions.cpu().detach().numpy()

    return predictions

# Creating the Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,  # Function to be called
    inputs=gr.Textbox(label="Enter Text", placeholder="Type your text here..."),  # Input text box
    outputs=gr.Dataframe(label="Predictions", headers=["Class 0", "Class 1"]),  # Displaying predictions as a dataframe
    title="Text Sentiment Prediction",  # Title of the interface
    description="Enter text to predict sentiment using a custom pre-trained model.",  # Description
    examples=[["This is a great movie!"], ["I really dislike this product."]]  # Example inputs
)

# Launch the interface
interface.launch()


OSError: Can't load tokenizer for 'Final_CustomModel_toxic_feb5'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Final_CustomModel_toxic_feb5' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

#SequenceClassifierOutput(loss=None, logits=tensor([[-3.5817,  3.5151]], device='cuda:0', grad_fn=<AddmmBackward0>), #hidden_states=None, attentions=None)
#tensor([[8.2704e-04, 9.9917e-01]], device='cuda:0', #grad_fn=<SoftmaxBackward0>)
#array([[0.00082704, 0.999173  ]], dtype=float32)



Result:
The probabilities for the two classes are:
#Class 0: 0.00082704 (~0.08%)
#Class 1: 0.999173 (~99.92%)

```
# This is formatted as code
```



In [ ]:
trainer.save_model('Final_CustomModel_toxic_feb5')


In [ ]:
trainer.save_model('/content/drive/MyDrive/dataset/Final_CustomModel_toxic_feb5')
model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/dataset/Final_CustomModel_toxic_feb5")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("Final_CustomModel_toxic_feb5")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# text = "That was good point"
text = "labeling an anarchist leader is very ridiculous"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.0017842, 0.9982158]], dtype=float32)